<a href="https://colab.research.google.com/github/EdithPraiselin/-SENTIMENT-ANALYSIS-WITH-NLP/blob/main/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tarfile

tar_path = "/content/drive/My Drive/dataset1.tar.gz"  # Update path
extract_path = "/content/dataset"

with tarfile.open(tar_path, "r:gz") as tar_ref:
    tar_ref.extractall(extract_path)

print("Files extracted successfully!")



Files extracted successfully!


In [ ]:
import os
import pandas as pd

# Function to load files from a folder
def load_reviews_from_folder(folder, label):
    reviews = []
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        with open(file_path, "r", encoding="utf-8") as file:
            reviews.append(file.read())
    return pd.DataFrame({"review": reviews, "sentiment": label})

# Define dataset paths
train_pos_path = "/content/dataset/aclImdb/train/pos"
train_neg_path = "/content/dataset/aclImdb/train/neg"
test_pos_path = "/content/dataset/aclImdb/test/pos"
test_neg_path = "/content/dataset/aclImdb/test/neg"

# Load data
train_pos = load_reviews_from_folder(train_pos_path, 1)  # Positive = 1
train_neg = load_reviews_from_folder(train_neg_path, 0)  # Negative = 0
test_pos = load_reviews_from_folder(test_pos_path, 1)
test_neg = load_reviews_from_folder(test_neg_path, 0)

# Combine positive and negative reviews
df_train = pd.concat([train_pos, train_neg]).sample(frac=1).reset_index(drop=True)  # Shuffle
df_test = pd.concat([test_pos, test_neg]).sample(frac=1).reset_index(drop=True)

# Display sample data
print(df_train.head())
print(df_test.head())


                                              review  sentiment
0  Viggo Mortensen stars as a new inmate of a hau...          1
1  it was the worst ending i have ever seen if so...          0
2  Ok, so, this is coming a few weeks late, but i...          1
3  Since Educating Rita, Julie Walters has been o...          1
4  This is an EXCELLENT example of early Bette Da...          1
                                              review  sentiment
0  The Vietnam War era is certainly far before my...          1
1  I realize why people hate this film. And, I ha...          0
2  This piece of crap doesn't worth a critical re...          0
3  This movie is funny if you're the gentleman wh...          0
4  Great premise, poor execution. Cast of great a...          0


In [ ]:
import os
import pandas as pd
import numpy as np
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from google.colab import drive

# Define dataset paths
train_pos_path = "/content/dataset/aclImdb/train/pos"
train_neg_path = "/content/dataset/aclImdb/train/neg"
test_pos_path = "/content/dataset/aclImdb/test/pos"
test_neg_path = "/content/dataset/aclImdb/test/neg"

# Function to read text files and assign labels
def load_reviews_from_folder(folder, label):
    reviews = []
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        with open(file_path, "r", encoding="utf-8") as file:
            reviews.append(file.read())
    return pd.DataFrame({"review": reviews, "sentiment": label})

# Load and label train/test data
train_pos = load_reviews_from_folder(train_pos_path, 1)  # Positive = 1
train_neg = load_reviews_from_folder(train_neg_path, 0)  # Negative = 0
test_pos = load_reviews_from_folder(test_pos_path, 1)
test_neg = load_reviews_from_folder(test_neg_path, 0)

# Combine positive and negative reviews
df_train = pd.concat([train_pos, train_neg]).sample(frac=1).reset_index(drop=True)
df_test = pd.concat([test_pos, test_neg]).sample(frac=1).reset_index(drop=True)

# Text Cleaning Function
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(f"[{string.punctuation}]", "", text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)  # Remove numbers
    return text.strip()

# Apply cleaning function
df_train['cleaned_review'] = df_train['review'].apply(clean_text)
df_test['cleaned_review'] = df_test['review'].apply(clean_text)

# Convert text to TF-IDF features
vectorizer = TfidfVectorizer(max_features=5000)  # Limit features for efficiency
X_train_tfidf = vectorizer.fit_transform(df_train['cleaned_review'])
X_test_tfidf = vectorizer.transform(df_test['cleaned_review'])

# Train Logistic Regression Model
model = LogisticRegression()
model.fit(X_train_tfidf, df_train['sentiment'])

# Predictions
y_pred = model.predict(X_test_tfidf)

# Evaluate Model
print("Accuracy:", accuracy_score(df_test['sentiment'], y_pred))
print(classification_report(df_test['sentiment'], y_pred))



Accuracy: 0.88232
              precision    recall  f1-score   support

           0       0.88      0.88      0.88     12500
           1       0.88      0.89      0.88     12500

    accuracy                           0.88     25000
   macro avg       0.88      0.88      0.88     25000
weighted avg       0.88      0.88      0.88     25000

